In [1]:
from transformers import DecisionTransformerModel, DecisionTransformerConfig
import numpy as np
import torch

device = "cpu"

scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 3600 / scale  # evaluation conditioning targets, 3600 is reasonable from the paper LINK
state_dim = (3, 32, 32)
state_dim_flatten = state_dim[0]*state_dim[1]*state_dim[2]
act_dim = 6

/home/pedron18/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
n_positions=1024*4
max_ep_len=100 # no impact?

t = int(n_positions/3) # number of elements in sequence
b = 1 # batch 

target_return = torch.rand((b, t, 1), device=device, dtype=torch.float32) # expected future return
states = torch.rand((b, t, state_dim_flatten), device=device, dtype=torch.float32)#.reshape(1, 1, state_dim_flatten) #
actions = torch.rand((b, t, act_dim), device=device, dtype=torch.float32)

rewards = torch.rand((b,2, t), device=device, dtype=torch.float32) # doesnt matter idk why exists
timesteps = torch.randint_like(torch.zeros((b, t)), device=device, low = 100, high = 102, dtype=torch.long) # integer what timestep we on

attention_mask = torch.ones((b, t), device=device, dtype=torch.long) # this is the default

print("target_return",target_return.shape)
print("states",states.shape)
print("actions",actions.shape)
print("rewards",rewards.shape)
print("timesteps", timesteps.shape)
print("attention_mask", attention_mask.shape)

config = DecisionTransformerConfig(state_dim_flatten, act_dim, max_ep_len=max_ep_len,n_positions=n_positions)


model = DecisionTransformerModel(config).to(device)
model.eval()
with torch.no_grad():
        state_preds, action_preds, return_preds = model(states=states,
                actions=actions,
                rewards=rewards,
                returns_to_go=target_return,
                timesteps=timesteps,
                attention_mask=attention_mask,
                return_dict=False)

        #print("argmax", torch.argmax(action_preds).item())
        print("state_preds",state_preds.shape)
        print("action_preds",action_preds.shape)
        print("return_preds", return_preds.shape)


        print("state_preds", state_preds[0])
        print("action_preds", action_preds[0])
        print("return_preds", return_preds[0])

target_return torch.Size([1, 1365, 1])
states torch.Size([1, 1365, 3072])
actions torch.Size([1, 1365, 6])
rewards torch.Size([1, 2, 1365])
timesteps torch.Size([1, 1365])
attention_mask torch.Size([1, 1365])
state_preds torch.Size([1, 1365, 3072])
action_preds torch.Size([1, 1365, 6])
return_preds torch.Size([1, 1365, 1])
state_preds tensor([[-7.5244e-02,  5.6634e-01, -3.4422e-04,  ..., -1.0548e-01,
         -4.8773e-02, -7.8832e-02],
        [-3.3674e-01,  2.3676e-01, -8.1974e-02,  ..., -1.1807e-01,
         -1.0754e-01,  7.4825e-02],
        [-2.9564e-02,  5.4737e-02,  7.6918e-02,  ..., -4.3481e-04,
          1.1208e-01,  5.5517e-02],
        ...,
        [-2.1386e-01,  3.9948e-01,  1.5797e-01,  ..., -2.0210e-02,
         -5.6381e-02,  1.3957e-01],
        [-3.5902e-01,  3.6190e-02,  1.8222e-01,  ...,  1.4219e-01,
         -1.0479e-01,  2.9675e-01],
        [-1.4891e-01,  2.5938e-01,  7.2407e-02,  ...,  1.3579e-01,
          7.0109e-02,  1.2475e-01]])
action_preds tensor([[ 0.1855, 

In [ ]:
import random

population = [0, 1, 2, 3, 4]
ls = random.sample(population,5)

print("Pure random: ", ls)

max_sequence = 4

def sample_with_order(population, max_sequence_length):
    """
        Samples [1,population] elements from a sequence mantaining the order
    """
    k = random.randint(1, max_sequence_length)
    k_start = random.randint(0, len(population)-k)
    return population[k_start: k_start+k]


print(sample_with_order(population, max_sequence))

   

Pure random:  [0, 1, 4, 2, 3]
[1, 2, 3, 4]


In [ ]:
target_return = torch.ones(1, device=device, dtype=torch.float32).reshape(100, 1)

RuntimeError: shape '[100, 1]' is invalid for input of size 1